# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = True
save_masks = False
print_loss = True
output_tensors = False
EPOCHS = 3
iteration = "-11"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 50
#val_batch_size = 100
num_batch = 2000 #25000 #11112
val_batch = 2
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    # time
    since = time.time()
    best_model = None
    best_loss = math.inf
    
    outputs_dir = "data/outputs/results/" + str(datetime.now().date())
    os.makedirs(outputs_dir, exist_ok=True)
    
    # logs
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    batch_log_name = str(datetime.now().date()) + iteration
    epoch_log_name = "epoch - "+ str(datetime.now().date()) + iteration
    
    # model and sigmoid function
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
            
        epoch_loss = 0
        
        #Training    
        model.train()
        phase = 'train'
        for batch in range(num_batch):
            if print_loss:
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            running_loss = 0.0
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)

            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                batch_log.write(log_dir, batch_log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= num_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, val_batch), flush=True)
            
            #BATCH TUPLE
#           inputs, labels, paths = next(iter(train_dataloaders))
            inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)

            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                batch_log.write(log_dir, batch_log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
            
            if save_masks and phase == 'val':
                outputs = sigmoid_function(outputs)
                j = 0
                for locations in paths:
                    img = Image.open(locations)
#                     num = locations.replace("./data/train/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
                    for threshold in [0.05, .25, .5, .75]:
                        img = data_factory.showMaskOnImage(img,outputs[j], threshold)
                        img.save(outputs_dir  + "/train-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
                    j += 1
                
        val_loss /= val_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, epoch_log_name)    

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model, LOAD_LOCATION)
            torch.save(model, SAVE_LOCATION)
            torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
        
    #save current model
    torch.save(model,SAVE_LOCATION)
    batch_log.write(log_dir, batch_log_name)
    epoch_log.write(log_dir, epoch_log_name)
    
    
    time_elapsed = time.time() - since
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet1(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    print('Single convolution model, no weights', flush=True)
#     model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
#     print('initializing model with random weights', flush=True)
#     torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)
#     torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  

Single convolution model with weights


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/3
Batch 1/2000
loss: 0.6952131390571594
---------------
Batch 2/2000
loss: 0.6931377053260803
---------------
Batch 3/2000
loss: 0.6962087750434875
---------------
Batch 4/2000
loss: 0.6894675493240356
---------------
Batch 5/2000
loss: 0.6921644806861877
---------------
Batch 6/2000
loss: 0.6890307664871216
---------------
Batch 7/2000
loss: 0.6965363621711731
---------------
Batch 8/2000
loss: 0.6911460757255554
---------------
Batch 9/2000
loss: 0.6925536394119263
---------------
Batch 10/2000
loss: 0.6931202411651611
---------------
Batch 11/2000
loss: 0.6937384009361267
---------------
Batch 12/2000
loss: 0.6934319734573364
---------------
Batch 13/2000
loss: 0.6901801824569702
---------------
Batch 14/2000
loss: 0.6896922588348389
---------------
Batch 15/2000
loss: 0.6902526021003723
---------------
Batch 16/2000
loss: 0.6902587413787842
---------------
Batch 17/2000
loss: 0.6908583641052246
---------------
Batch 18/2000
loss: 0.6913305521011353
---------------
Batch 19/

---------------
Batch 150/2000
loss: 0.6904822587966919
---------------
Batch 151/2000
loss: 0.6911653876304626
---------------
Batch 152/2000
loss: 0.6915717720985413
---------------
Batch 153/2000
loss: 0.6921656131744385
---------------
Batch 154/2000
loss: 0.690434455871582
---------------
Batch 155/2000
loss: 0.6918759942054749
---------------
Batch 156/2000
loss: 0.6906991004943848
---------------
Batch 157/2000
loss: 0.6909192204475403
---------------
Batch 158/2000
loss: 0.6908412575721741
---------------
Batch 159/2000
loss: 0.6907674074172974
---------------
Batch 160/2000
loss: 0.6899275779724121
---------------
Batch 161/2000
loss: 0.692944347858429
---------------
Batch 162/2000
loss: 0.6904479265213013
---------------
Batch 163/2000
loss: 0.6888794302940369
---------------
Batch 164/2000
loss: 0.6914562582969666
---------------
Batch 165/2000
loss: 0.6915059685707092
---------------
Batch 166/2000
loss: 0.6869116425514221
---------------
Batch 167/2000
loss: 0.69276082515

---------------
Batch 297/2000
loss: 0.6916022300720215
---------------
Batch 298/2000
loss: 0.691688597202301
---------------
Batch 299/2000
loss: 0.6911064386367798
---------------
Batch 300/2000
loss: 0.690422534942627
---------------
Batch 301/2000
loss: 0.6906306743621826
---------------
Batch 302/2000
loss: 0.691393256187439
---------------
Batch 303/2000
loss: 0.6900241374969482
---------------
Batch 304/2000
loss: 0.6906141638755798
---------------
Batch 305/2000
loss: 0.6919064521789551
---------------
Batch 306/2000
loss: 0.6946898698806763
---------------
Batch 307/2000
loss: 0.6942352056503296
---------------
Batch 308/2000
loss: 0.6917351484298706
---------------
Batch 309/2000
loss: 0.6945206522941589
---------------
Batch 310/2000
loss: 0.6903976798057556
---------------
Batch 311/2000
loss: 0.6911138892173767
---------------
Batch 312/2000
loss: 0.692645251750946
---------------
Batch 313/2000
loss: 0.6916627287864685
---------------
Batch 314/2000
loss: 0.6912527084350

---------------
Batch 444/2000
loss: 0.6920588612556458
---------------
Batch 445/2000
loss: 0.6923004984855652
---------------
Batch 446/2000
loss: 0.688197135925293
---------------
Batch 447/2000
loss: 0.6929155588150024
---------------
Batch 448/2000
loss: 0.6912132501602173
---------------
Batch 449/2000
loss: 0.690015971660614
---------------
Batch 450/2000
loss: 0.6919109225273132
---------------
Batch 451/2000
loss: 0.6909030079841614
---------------
Batch 452/2000
loss: 0.6899049878120422
---------------
Batch 453/2000
loss: 0.6912310719490051
---------------
Batch 454/2000
loss: 0.6909970045089722
---------------
Batch 455/2000
loss: 0.6921473741531372
---------------
Batch 456/2000
loss: 0.6895435452461243
---------------
Batch 457/2000
loss: 0.6916817426681519
---------------
Batch 458/2000
loss: 0.6928116083145142
---------------
Batch 459/2000
loss: 0.6910357475280762
---------------
Batch 460/2000
loss: 0.6931276917457581
---------------
Batch 461/2000
loss: 0.69082051515

---------------
Batch 591/2000
loss: 0.6892346739768982
---------------
Batch 592/2000
loss: 0.6894737482070923
---------------
Batch 593/2000
loss: 0.6896748542785645
---------------
Batch 594/2000
loss: 0.6888699531555176
---------------
Batch 595/2000
loss: 0.6905316114425659
---------------
Batch 596/2000
loss: 0.6895222663879395
---------------
Batch 597/2000
loss: 0.6926161050796509
---------------
Batch 598/2000
loss: 0.6940469145774841
---------------
Batch 599/2000
loss: 0.6894358992576599
---------------
Batch 600/2000
loss: 0.692290186882019
---------------
Batch 601/2000
loss: 0.6918743252754211
---------------
Batch 602/2000
loss: 0.6944149732589722
---------------
Batch 603/2000
loss: 0.691785454750061
---------------
Batch 604/2000
loss: 0.6905772686004639
---------------
Batch 605/2000
loss: 0.6917024850845337
---------------
Batch 606/2000
loss: 0.6913580894470215
---------------
Batch 607/2000
loss: 0.6919372081756592
---------------
Batch 608/2000
loss: 0.69022631645

...

loss: 0.6934167146682739
---------------
Batch 1393/2000
loss: 0.6904939413070679
---------------
Batch 1394/2000
loss: 0.6918709874153137
---------------
Batch 1395/2000
loss: 0.6923218965530396
---------------
Batch 1396/2000
loss: 0.6925427317619324
---------------
Batch 1397/2000
loss: 0.6900261044502258
---------------
Batch 1398/2000
loss: 0.6906288266181946
---------------
Batch 1399/2000
loss: 0.690205991268158
---------------
Batch 1400/2000
loss: 0.6914517283439636
---------------
Batch 1401/2000
loss: 0.6886858344078064
---------------
Batch 1402/2000
loss: 0.689971923828125
---------------
Batch 1403/2000
loss: 0.6920892596244812
---------------
Batch 1404/2000
loss: 0.687728226184845
---------------
Batch 1405/2000
loss: 0.6905616521835327
---------------
Batch 1406/2000
loss: 0.6897527575492859
---------------
Batch 1407/2000
loss: 0.6931257247924805
---------------
Batch 1408/2000
loss: 0.6887367367744446
---------------
Batch 1409/2000
loss: 0.69083571434021
-----------

---------------
Batch 1537/2000
loss: 0.691024124622345
---------------
Batch 1538/2000
loss: 0.6903172135353088
---------------
Batch 1539/2000
loss: 0.6911051869392395
---------------
Batch 1540/2000
loss: 0.6904973387718201
---------------
Batch 1541/2000
loss: 0.6891071200370789
---------------
Batch 1542/2000
loss: 0.6920881271362305
---------------
Batch 1543/2000
loss: 0.6914768218994141
---------------
Batch 1544/2000
loss: 0.6930302381515503
---------------
Batch 1545/2000
loss: 0.6898349523544312
---------------
Batch 1546/2000
loss: 0.6892564296722412
---------------
Batch 1547/2000
loss: 0.6904327869415283
---------------
Batch 1548/2000
loss: 0.690704345703125
---------------
Batch 1549/2000
loss: 0.6919800639152527
---------------
Batch 1550/2000
loss: 0.6915102601051331
---------------
Batch 1551/2000
loss: 0.6913076043128967
---------------
Batch 1552/2000
loss: 0.6925961375236511
---------------
Batch 1553/2000
loss: 0.6937834024429321
---------------
Batch 1554/2000
l

Batch 1681/2000
loss: 0.6908064484596252
---------------
Batch 1682/2000
loss: 0.6922451853752136
---------------
Batch 1683/2000
loss: 0.6911477446556091
---------------
Batch 1684/2000
loss: 0.6921292543411255
---------------
Batch 1685/2000
loss: 0.6926975846290588
---------------
Batch 1686/2000
loss: 0.6922293305397034
---------------
Batch 1687/2000
loss: 0.6904818415641785
---------------
Batch 1688/2000
loss: 0.6915232539176941
---------------
Batch 1689/2000
loss: 0.6908427476882935
---------------
Batch 1690/2000
loss: 0.6904842853546143
---------------
Batch 1691/2000
loss: 0.6900246739387512
---------------
Batch 1692/2000
loss: 0.6898742914199829
---------------
Batch 1693/2000
loss: 0.6913939714431763
---------------
Batch 1694/2000
loss: 0.6902564764022827
---------------
Batch 1695/2000
loss: 0.6903856992721558
---------------
Batch 1696/2000
loss: 0.693183958530426
---------------
Batch 1697/2000
loss: 0.6918948888778687
---------------
Batch 1698/2000
loss: 0.69252938

Batch 1825/2000
loss: 0.6887659430503845
---------------
Batch 1826/2000
loss: 0.6915488839149475
---------------
Batch 1827/2000
loss: 0.6888641715049744
---------------
Batch 1828/2000
loss: 0.6918891072273254
---------------
Batch 1829/2000
loss: 0.6942785382270813
---------------
Batch 1830/2000
loss: 0.6924652457237244
---------------
Batch 1831/2000
loss: 0.6891624927520752
---------------
Batch 1832/2000
loss: 0.6896851658821106
---------------
Batch 1833/2000
loss: 0.6926324367523193
---------------
Batch 1834/2000
loss: 0.6906552910804749
---------------
Batch 1835/2000
loss: 0.6890507340431213
---------------
Batch 1836/2000
loss: 0.6892920136451721
---------------
Batch 1837/2000
loss: 0.6916875243186951
---------------
Batch 1838/2000
loss: 0.6891985535621643
---------------
Batch 1839/2000
loss: 0.6921476721763611
---------------
Batch 1840/2000
loss: 0.6901870965957642
---------------
Batch 1841/2000
loss: 0.6915644407272339
---------------
Batch 1842/2000
loss: 0.6909928

Batch 1969/2000
loss: 0.6913211941719055
---------------
Batch 1970/2000
loss: 0.6925904750823975
---------------
Batch 1971/2000
loss: 0.6931231617927551
---------------
Batch 1972/2000
loss: 0.6908207535743713
---------------
Batch 1973/2000
loss: 0.6886575222015381
---------------
Batch 1974/2000
loss: 0.6920545101165771
---------------
Batch 1975/2000
loss: 0.6938793063163757
---------------
Batch 1976/2000
loss: 0.6931686401367188
---------------
Batch 1977/2000
loss: 0.690727710723877
---------------
Batch 1978/2000
loss: 0.6880336999893188
---------------
Batch 1979/2000
loss: 0.6914499998092651
---------------
Batch 1980/2000
loss: 0.6917332410812378
---------------
Batch 1981/2000
loss: 0.6929726600646973
---------------
Batch 1982/2000
loss: 0.688583254814148
---------------
Batch 1983/2000
loss: 0.6899600028991699
---------------
Batch 1984/2000
loss: 0.6916214227676392
---------------
Batch 1985/2000
loss: 0.6883574724197388
---------------
Batch 1986/2000
loss: 0.692260205

# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [8]:
#save state model as model
model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
model.load_state_dict(torch.load(LOAD_LOCATION+'_test-2019-04-23-3'))
torch.save(model, LOAD_LOCATION)